In [60]:
import requests
import json

In [61]:
def get_results(data, key):
    result = []
    current_page = data
    has_next = True
    
    while(has_next):
        for data_point in current_page['results']:
            result.append(data_point)

        if current_page['next'] == None:
            has_next = False
        else:
            response = requests.get(current_page['next'])
            current_page = response.json()

    return result

In [62]:
response = requests.get("https://swapi.dev/api/?format=json")
root = response.json()

og_data = {}

for key in root:
    response = requests.get(root[key] + '/?format=json')
    og_data[key] = get_results(response.json(), key)
    print(key, 'done')

with open("result2.json", "w") as write_file:
    json.dump(og_data, write_file, indent=4)


people done
planets done
films done
species done
vehicles done
starships done


In [63]:
data = og_data.copy()

for v in data.values():
    for item in v:
        for key, value in item.items():
            original_key = key

            if key == 'pilots' or key == 'characters' or key == 'residents':
                original_key = 'people'

            if type(value) == list and original_key in data.keys():
                items = []
                for val in value:
                    for root_data in data[original_key]:
                        if root_data['url'] == val:
                            if ('name' in root_data):
                                items.append(root_data['name'])
                            else:
                                items.append(root_data['title'])

                            break
                item[key] = items

            if key == 'homeworld':
                original_key = 'planets'
                for root_data in data[original_key]:
                    if root_data['url'] == value:
                        item[key] = root_data['name']
                        break


with open("result.json", "w") as write_file:
    json.dump(data, write_file, indent=4)